# Information-Theoretic Security (examples and exercises)

## Definitions of Information Entropy

### Source Entropy

Given a message $m$ taken from a set of messages $\{m_1, \dots, m_N\}$ with probabilities $p(m_i)\ 1\leq i \leq N$, the source entropy is

$H(m) = -\sum_{i=1}^N p_i \log_2 p_i$

Source entropy measures the minimum number of bits necessary to encode the source messages.

### Joint Entropy

Given a message $m$ taken from a set of messages $\{m_i\}$ with probabilities $p(m_i)$ and a message $c$ taken from a set of messages $\{c_j\}$  with probabilities $p(c_j)$ and joint probabilities $p(m_i,c_j)$, the joint entropy is

$H(m,c) = - \sum_{i,j} p(m_i,c_j) \log_2 p(m_i,c_j)$

### Conditional Entropy

Given a message $m$ taken from a set of messages $\{m_i\}$ with probabilities $p(m_i)$ and a ciphertext $c$ taken from a set of messages $\{c_j\}$  with probabilities $p(c_j)$ and joint probabilities $p(m_i,c_j)$, the conditional entropy is

$H(c|m) = \sum_i H(c|m_i) \Pr[m_i]$

### Chain Rule

$H(m,c) = H(m) + H(c|m) = H(c) + H(m|c)$

### Bayes' Theorem

$H(m|c) = H(c|m) - H(c) + H(m)$

### Mutual Information

$I(m;c) = H(m)-H(m|c) = H(c)-H(c|m)$

Mutual information is equal to zero if and only if the message and the ciphertext  are independent.

## Exercise 1

Assume that the message space is $M = \{\texttt{A}, \texttt{Z} \}$.
Alice chooses $m=\texttt{A}$ with probability 0.7 and $m=\texttt{Z}$ with probability 0.3. Alice encrypts the message with some random key.

Eve, an eavesdropper, sees $c=\texttt{B}$. What can she say about the plaintext?
What is $I(m;c)$?

**Solution (analytical)**

The entropy of the message source is:

$H(m) =
- \Pr[m=\texttt{A}] \log_2 \Pr[m=\texttt{A}]
- \Pr[m=\texttt{Z}] \log_2 \Pr[m=\texttt{Z}]	
= 0.88\,\mathrm{bit}$

Given a plaintext, all ciphertexts are equally likely, so 

$H(c|m) = 26 \times \frac{1}{26}\log_2 26 = 4.7\,\mathrm{bit}$

We already know that

$\Pr[c=\texttt{A}]=\dots=\Pr[c=\texttt{Z}]=1/26$

thus $H(c)=4.7\,\mathrm{bit}$.

Therefore:

$I(m;c) = H(c) - H(c|m) = 0$

No information is learned.

**Solution (empirical)**

In [1]:
from random import random, randint
import numpy as np

In [3]:
# calculates the entropy of a list of elements
# builds the empyrical pdf of the list
# then applies the definition
def entropy_of_list(l):
    histogram = {x:(l.count(x)+0.0)/len(l) for x in l}
    return sum([-x*log(x,2) for x in histogram.values()])

In [ ]:
# Instantiate a shift cipher over the alphabet
S = ShiftCryptosystem(AlphabeticStrings());

# Define the random key generator
def keygen():
    return randint(0,25)

In [2]:
# message source
def message1():
    if random()<=0.7:
        return 'A'
    else:
        return 'Z'

In [11]:
# generate several random messages 
m = [S.encoding(message1()) for i in range(1000)]

# print empirical entropy of the message source
print "Empirical Entropy of source is:", entropy_of_list(m)

Empirical Entropy of source is: 0.893173458377857


In [12]:
# Encrypt each message with a different random key
c = [ S.enciphering( keygen() , x ) for x in m ]

print "Empirical Entropy of ciphertext is:", entropy_of_list(c)

Empirical Entropy of ciphertext is: 4.68926722828748


## Exercise 2

Assume that $\Pr[m=\texttt{kim}]=0.5$, $\Pr[m=\texttt{ann}]=0.2$, and $\Pr[m=\texttt{boo}]=0.3$.

What is $I(m;c)$?

**Solution (analytical)** 
Analogous calculations can be done using information entropy.
The entropy of the message source is:

$H(m) = 1.49 bit$

Ciphertexts are not equally likely. There are only $26+26=52$ different CTs, half of which obtained from $\texttt{kim}$ and half from $\texttt{ann}$ and $\texttt{boo}$.

$H(c) = - 52 \frac{1}{52}\log_2\frac{1}{52} = 5.7 bit$

Given a plaintex, all ciphertexts are equally likely:
$H(c|m) = 4.7 bit$.

$I(m;c) = H(c) -H(c|m) = 1 bit$
meaning that knowledge of $c$ gives us 1 bit of information.

What does it mean that mutual information is 1 bit? It means that there exists an agorithm that can answer a yes/no question about the plaintext by just looking to the ciphertext.

This does not tell us what is the algorithm to obtain this bit. Neither whether this algorithm is efficient. It only tells us that this algorithm exists.
The following is just an example:

```
def plaintext_is_kim(ciphertext):
    if last two letters of ciphertext are the same:
        return false # plaintext is not kim
    else
        return true # plaintext is kim
    end if
```

**Solution (empirical)**

In [14]:
# generate random three letter messages
def message2():
    r = random() 
    if r <= 0.5:
        return 'KIM'
    if r <= 0.7:
        return 'ANN'
    else:
        return 'BOO'

In [15]:
# generate random messages 
m = [S.encoding(message2()) for i in range(1000)]

# print empirical entropy of the message source
print "Empirical Entropy of source is:", entropy_of_list(m)

Empirical Entropy of source is: 1.47717700758957


In [16]:
# Encrypt each message with a random key
c=[S.enciphering( keygen(), x) for x in m]

print "Empirical Entropy of ciphertext is:", entropy_of_list(c)

Empirical Entropy of ciphertext is: 5.67313984293835


## Exercise 3

Consider a Vigènere cipher. Messages are 4-character sequences of letters. Passwords are $n$-character long.

Assume that there are three possible messages, with different probabilities.

| Message | Probability |
|---|---|
| $m_1$ = BEBA | 0.5 |
| $m_2$ = DEDA | 0.3 |
| $m_3$ = CFCB | 0.2 |

1. Calculate the entropy of the source.
2. Calculate the entropy of the key with $n=1,2,4$.
3. Calculate the entropy of the ciphertext with $n=1,2,4$.
4. Calculate the information leak of the ciphertext with $n=1,2,4$
5. Is the cipher of this exercise perfectly secret?

### Solution

#### Part 1

$H(M) = -0.5\log_2 0.5 -0.3\log_2 0.3 -0.2\log_2 0.2 = 1.48$

#### Part 2

| $n$ | $H(K)$ |
| --- | ------ |
| 1   | 4.7    |
| 2   | 9.4    |
| 4   | 18.8   |


#### Part 3

Let call $C_i$ the set of possible ciphertexts for message $m_i$.

With $n=1$, we have
* $C_1 = \{ \text{BEBA}, \text{CFCB}, \text{DGDC}, \dots \}$
* $C_2 = \{ \text{DEDA}, \text{EFEB}, \text{FGFB}, \dots \}$
* $C_3 = \{ \text{CFCB}, \text{DGDC}, \text{EHED}, \dots \}$

So $C_1 = C_3$, while $C_2$ is distinct. Thus, we have $2\times26=52$ ciphertexts. The first set has probability $0.5+0.2=0.7$. The second set has probability $0.3$.

$H(C)=
    -26\times 0.7/26\log_2 0.7/26
    -26\times 0.3/26\log_2 0.3/26
    = 5.58
$

With $n=2$, we have again $C_1 = C_2 = C_3$. In fact, the key $(2,0)$ can map BEBA into DEDA, while the key $(1,1)$ can map BEBA into CFCB. Consequently all the messages generate the same $26^2=676$ ciphertexts.

$H(C)= -676 / 676 \log_2(1/676)=9.4$

With $n=4$, we have $C_1 = C_2 = C_3$. Thus, we have $26^4=456976$ equally likely ciphertexts.

$H(C)= 26^4 / 26^4 \log_2(1/26^4)=18.8$

#### Part 4

With $n=1$

$H(C|M)=H(C_1)\Pr(m_1) + H(C_2)\Pr(m_2) + H(C_3)\Pr(m_3)= 4.7$

Then
$
	I(M;C) =
	H(C)-H(C|M) = 5.58 - 4.7 = 0.88
$

With $n=2$

$
	H(C|M)=H(C_1)\Pr(m_1) + H(C_2)\Pr(m_2+m_3)= 9.4
$

Then
$
	I(M;C) =
	H(C)-H(C|M) = 9.4-9.4 = 0
$

With $n=4$

$
	H(C|M)=H(C_1)\Pr(m_1+m_2+m_3)= 18.8
$

Then
$
	I(M;C) =
	H(C)-H(C|M) = 18.8-18.8=0
$

#### Part 5
If these three are the only possible messages, then with $n=2$ and $n=4$ we have perfect secrecy.